In [20]:
import googlemaps
import os
import pandas as pd
import geopandas as gpd
import numpy as np
import networkx as nx

# Specify the path of your own python shared library
os.environ["PROJ_LIB"] = r"C:\\Users\\SQwan\\miniconda3\\Library\\share"
from mpl_toolkits.basemap import Basemap
from datetime import datetime

# Code for Estimating Travel Time via Google Maps API
Google Maps Platform features a recurring \$200 monthly credit. Before using google maps API, follow this [link](https://developers.google.com/maps/documentation/directions/quickstart "The Directions API quickstart") to obtain an API key. 

Required packages: `googlemaps`, `pandas`, `geopandas`, `networkx`.<br>

`API_KEY`: After having an API key, copy it to variable `API_KEY`. <br>

`MODE`: Travel mode used for acquring Google Maps directions. Available travel modes can be found [here](https://developers.google.com/maps/documentation/directions/get-directions#TravelModes). <br>

`TIME`: Departure time should be no later than the current time. Specifiy the time in the format of "Y-M-D-hr-min-sec". <br>

`shapefile_loc`: Shapefile "destinations_in_region.shp" location. <br>

Other parameters of Directions API can be found in the [document](https://developers.google.com/maps/documentation/directions/get-directions).

In [2]:
# copy your google map API key
API_KEY = ""
# travel mode
MODE = "transit"
# departure time needs to be no later than current time
TIME = datetime(2021, 7, 1, 12, 0, 0)
# shapefile location
shapefile_loc = r"Data\\shapefile\\destinations_in_region\\destinations_in_region.shp"

## Import shapefile

In [3]:
poly_df = gpd.read_file(shapefile_loc)
poly_df = poly_df.loc[poly_df.in_region == 1]
poly_df["zone_id"] = np.arange(len(poly_df))
# Find boundary of the shapefile
poly_df_summary = poly_df.geometry.bounds
lolon, uplon, lolat, uplat = (
    poly_df_summary.minx.min(),
    poly_df_summary.maxx.max(),
    poly_df_summary.miny.min(),
    poly_df_summary.maxy.max(),
)
# Find centroids of BH shapefile
bm = Basemap(
    llcrnrlon=lolon,
    llcrnrlat=lolat,
    urcrnrlon=uplon,
    urcrnrlat=uplat,
    resolution="i",
    projection="tmerc",
    lat_0=42.07,
    lon_0=-86.45,
)

# copy poly to new GeoDataFrame
centroid_df = poly_df.copy(deep=True).to_crs(epsg=3395)
# change the geometry to centroid of each polygon
centroid_df["centroid"] = centroid_df.centroid
centroid_df = centroid_df.set_geometry("centroid")
centroid_df = centroid_df.to_crs(poly_df.crs)
centroid_df.rename(columns={"ID": "GEOID"}, inplace=True)

# get utm coordinates for the centroids
c_lon, c_lat = bm(centroid_df.geometry.x, centroid_df.geometry.y, inverse=False)
# reverse lat lon order used for googlemaps API
lat_lon = np.c_[c_lat, c_lon]
station = dict()
for i, center in zip(centroid_df.zone_id, centroid_df.geometry):
    station[i] = dict()
    station[i]["lat_lon"] = [
        centroid_df.geometry.iloc[i].y,
        centroid_df.geometry.iloc[i].x,
    ]
    station[i]["utm"] = lat_lon[i]
    station[i]["geoid"] = centroid_df.GEOID.iloc[i]
    neighbors = poly_df.zone_id[~poly_df.disjoint(poly_df.geometry.iloc[i])]
    # exclude self node in neighbours
    station[i]["neighbours"] = [n for n in neighbors if n != i]

In [16]:
num_station = len(station)
gmaps = googlemaps.Client(key=API_KEY)

# construct graphs of travel time and travel distance
G_d = nx.Graph()
G_t = nx.Graph()
G_d.add_nodes_from(np.arange(num_station))
G_t.add_nodes_from(np.arange(num_station))

for i in range(num_station):
    for j in station[i]["neighbours"]:
        if i < j:
            direction_result = gmaps.directions(
                tuple(station[i]["lat_lon"]),
                tuple(station[j]["lat_lon"]),
                mode=MODE,
                avoid="ferries",
                departure_time=TIME,
            )

            if direction_result:
                w_t = np.round(
                    direction_result[0]["legs"][0]["duration"][u"value"] / 60.0,
                    decimals=2,
                )
                w_d = np.round(
                    direction_result[0]["legs"][0]["distance"][u"value"] * 0.000621371,
                    decimals=2,
                )
            else:
                w_t = 9999
                w_d = 9999

            G_t.add_edge(i, j, weight=w_t)
            G_d.add_edge(i, j, weight=w_d)

In [22]:
id_converter = {i: int(j) for (i, j) in zip(poly_df["zone_id"], poly_df["GEOID"])}

S = len(poly_df)

df_time = pd.DataFrame()
df_dist = pd.DataFrame()

for i in poly_df["zone_id"]:
    for j in poly_df["zone_id"]:
        if i == j:
            df_time.loc[id_converter[i], id_converter[j]] = 0
            df_dist.loc[id_converter[i], id_converter[j]] = 0
        else:
            sp_time = nx.shortest_path_length(G_t, source=i, target=j, weight="weight")
            df_time.loc[id_converter[i], id_converter[j]] = (
                sp_time if sp_time < 9999 else 9999
            )
            sp_dist = nx.shortest_path_length(G_d, source=i, target=j, weight="weight")
            df_dist.loc[id_converter[i], id_converter[j]] = (
                sp_dist if sp_dist < 9999 else 9999
            )

df_time.to_csv(r"Data\gm_transit_time_min.csv")
df_dist.to_csv(r"Data\gm_transit_dist_km.csv")